## Introduction
AutoScout Data Analysis Project, ikinci el araba fiyatlarına etki eden hususları inceleyerek iyi bir fiyat tahmini modülü oluşturabilmek için gerekli veriyi hazır hale getirebilmek ve sonuçta iyi bir fiyat tahmini yapabilmektir. Burada online satış yapan bir firmadan alınan ve 9 farklı araba modeline ait farklı ve dağınık bir veri seti mevcuttur. Burada yapılacak işlemler 3 aşamada değerlendirebiliriz. 

The project consists of 3 parts:
* First part is related with 'data cleaning'. It deals with Incorrect Headers, Incorrect Format, Anomalies, Dropping useless columns.
* Second part is related with 'filling data'. It deals with Missing Values. Categorical to numeric transformation is done.
* Third part is related with 'handling outliers of data' via Visualisation libraries. Some insights are extracted.

In this project, Python libraries such as Numpy, Pandas, Matplotlib, Seaborn, Scipy kullanarak hem dataset temizliği hem de elde ettiğimiz dataseti üzerinden analysis yapma imkanı bulacağız.

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_json('scout_car.json', lines=True, orient='records', convert_axes=True, dtype=True,convert_dates=True)
df.head(3)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sportback-1-4-tdi-s-tronic-xenon-navi-klima-diesel-b...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate control, Cruise control, Electrical side mirrors, ...","[Bluetooth, Hands-free equipment, On-board computer, Radio]","[Alloy wheels, Catalytic Converter, Voice Control]","[ABS, Central door lock, Daytime running lights, Driver-side airbag, Electronic stability contro...","[\n, Sicherheit:, , Deaktivierung für Beifahrer-Airbag, , ESC mit elektronischer Quersperre, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-8-tfsi-sport-gasoline-red-b2547f8a-e83f-6237-e053-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)], [4.7 l/100 km (country)]]",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, Hill Holder, Leather steering wheel, Lumbar suppor...","[Bluetooth, Hands-free equipment, On-board computer, Radio, Sound system]","[Alloy wheels, Sport seats, Sport suspension, Voice Control]","[ABS, Central door lock, Central door lock with remote control, Daytime running lights, Driver-s...","[\nLangstreckenfahrzeug daher die hohe Kilometerleistung. , Bremsen neu bei 62000 km. , Gut gepf...",[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sportback-1-6-tdi-s-tronic-einparkhilfe-plus-music-d...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+music,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)], [3.4 l/100 km (country)]]",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical side mirrors, Hill Holder, Leather steering wheel,...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running lights, Driver-side airbag, Electronic stability 

In [4]:
# df.tail(20)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
15899,https://www.autoscout24.com//offers/renault-espace-2-0-blue-dci-200-initiale-paris-edc-eu6-2-die...,Renault Espace,2.0 Blue dCi 200 Initiale Paris EDC (EU6.2),Van,41395,VAT deductible,"5,000 km",03/2019,1 previous owner,NaN,147 kW,"[, Demonstration, , Diesel]",\n1\n,NaN,NaN,"[\n48 months\n, \n139 g CO2/km (comb)\n]",NaN,NaN,[],\nRenault\n,"[\n, Espace, \n]",NaN,"[\n, 2019, \n]","[\n, Grey, \n]",[\nMetallic\n],NaN,"[\nFull leather, Brown\n]","[\n, Van, \n]",[\n5\n],[\n7\n],NaN,"[\n, Automatic, \n]","[\n1,997 cc\n]",[\n4\n],"[\n1,840 kg\n]",[\nfront\n],"[\n, Diesel, \n]","[[5.3 l/100 km (comb)], [6.2 l/100 km (city)], [4.7 l/100 km (country)]]",[\n139 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Air suspension, Armrest, Automatic climate control, Cruise control, Electrica...","[Bluetooth, Digital radio, Hands-free equipment, MP3, On-board computer, Sound system, USB]","[Alloy wheels, Catalytic Converter, Touch screen, Voice Control]","[ABS, Adaptive Cruise Control, Blind spot monitor, Driver-side airbag, Electronic stability cont...","[\n, ..::Renault Espace Initiale Paris Dci 200 EDC (Boite auto)::.., Véhicule de direction,état ...",NaN,[\n6\n],[\nBelgium\n],NaN,NaN,NaN,NaN,NaN,NaN
15900,https://www.autoscout24.com//offers/renault-espace-espace-initiale-paris-tce-225-edc-gpf-gasolin...,Renault Espace,ESPACE INITIALE PARIS TCe 225 EDC GPF,Van,41480,VAT deductible,"8,500 km",01/2019,None,NaN,165 kW,"[, Demonstration, , Super 95]",NaN,NaN,\nYes\n,NaN,NaN,"[\n, \n, \n175 g CO2/km (comb)\n]",[],\nRenault\n,"[\n, Espace, \n]",[\nC0306\n],"[\n, 2019, \n]","[\n, Black, \n]",[\nMetallic\n],[\nSchwarz\n],"[\nFull leather, Brown\n]","[\n, Van, \n]",[\n5\n],[\n7\n],[\n3333/BHJ\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",NaN,"[\n1,843 kg\n]",[\nfront\n],"[\n, Super 95, \n]","[[7.4 l/100 km (comb)], [9.6 l/100 km (city)], [6 l/100 km (country)]]",[\n175 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, Cruise control, Electrically adjustable seats, Ele...",[Sound system],[Alloy wheels],"[ABS, Adaptive headlights, Blind spot monitor, Central door lock, Daytime running lights, Driver...","[\n, Unfallfrei, Euro6, Klimaautomatik 2-fach, Fahrerairbag, Beifahrerairbag, Fenster-/Kopfairba...",[\n4 (Green)\n],NaN,[\nGermany\n],NaN,NaN,NaN,NaN,NaN,NaN
15901,https://www.autoscout24.com//offers/renault-espace-blue-dci-160cv-edc-executive-4control-nuova-d...,Renault Espace,"Blue dCi 160CV EDC Executive 4Control , NUOVA DA I",Van,41400,VAT deductible,10 km,06/2019,None,NaN,118 kW,"[, Pre-registered, , Diesel]",NaN,NaN,NaN,\n,NaN,NaN,[],\nRenault\n,"[\n, Espace, \n]",[\n2691300\n],"[\n, 2019, \n]","[\n, Grey, \n]",NaN,[\nGrigio\n],[\nFull leather\n],"[\n, Van, \n]",[\n5\n],[\n7\n],NaN,"[\n, Automatic, \n]","[\n1,997 cc\n]",[\n4\n],"[\n1,758 kg\n]",[\nfront\n],"[\n, Diesel, \n]","[[5.1 l/100 km (comb)], [5.7 l/100 km (city)], [4.8 l/100 km (country)]]",[\n135 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, Cruise control, Electrically adjustable seats, Ele...","[Bluetooth, Digital radio, Radio, USB]","[Alloy wheels, Touch screen]","[ABS, Blind spot monitor, Driver-side airbag, Electronic stability control, Emergency brake assi...","[\nDEK:[2691300], Renault Espace Blue dCi 160CV EDC Executive 4Control, Nuova da Immatricolare

In [4]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            15919 non-null  object 
 1   make_model                     15919 non-null  object 
 2   short_description              15873 non-null  object 
 3   body_type                      15859 non-null  object 
 4   price                          15919 non-null  int64  
 5   vat                            11406 non-null  object 
 6   km                             15919 non-null  object 
 7   registration                   15919 non-null  object 
 8   prev_owner                     9091 non-null   object 
 9   kW                             0 non-null      float64
 10  hp                             15919 non-null  object 
 11  Type                           15917 non-null  object 
 12  Previous Owners                9279 non-null  

# Part-1: Data Cleanings

### Veri setimizi incelemeye başlamadan önce ilk olarak boş bir dataframe oluşturuyoruz. Sırasıyla her bir feature inceleyerek bu dataframe ekleyecek ve işlemlerimizi bunun üzerinden yapacağız.

In [5]:
data  = pd.DataFrame()

### Veri setimizin sütunlarını tek tek inceleyeceğiz. Çünkü veri setindeki eksik verileri doldururken bunlardan faydalanabileceğimiz ve bize net bilgi verebilecek değerler olabilir. 

### İlk olarak "make_model" sütunu iki parçaya bölerek başlıyoruz. Dataframe içinde ilave olarak "make" ve "model" olmak üzere iki sütun mevcut. Bunları da duplicate olacağından dolayı sileceğiz.

In [6]:
temp = df["make_model"].copy()

In [7]:
data["make"] = temp.str.split(' ', expand=True)[0]

In [8]:
data["model"] = temp.str.split(' ', expand=True)[1]

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   make    15919 non-null  object
 1   model   15919 non-null  object
dtypes: object(2)
memory usage: 248.9+ KB


### Veri setimizde kullanmayacağımız feature incelesek dahi bunları yeni oluşturduğumuz dataframe dahil etmeyeceğimizden dolayı herhangi bir silme işlemi yapmayacağız. "url" sütunu fiyatlara etki eden bir bilgiye sahip olmadığından dolayı bunu kullanmayacağız. 

In [10]:
# df.drop(["url","make_model"], axis=1).head(2)  # düşülecek.

,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,Sportback 1.4 TDI S-tronic...,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particu...",\n2\n,"[\n06/2021\n, \n99 g CO2/k...","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate F...","[[3.8 l/100 km (comb)], [4...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest...","[Bluetooth, Hands-free equ...","[Alloy wheels, Catalytic C...","[ABS, Central door lock, D...","[\n, Sicherheit:, , Deakt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automat...","[Bluetooth, Hands-free equ...","[Alloy wheels, Sport seats...","[ABS, Central door lock, C...",[\nLangstreckenfahrzeug da...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
temp = df["body_type"].copy()  # 60 tane eksik veri var. 

In [11]:
temp.value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [12]:
data["body_type"] = temp

In [13]:
data[data["body_type"].notna()]

,make,model,body_type
0,Audi,A1,Sedans
1,Audi,A1,Sedans
2,Audi,A1,Sedans
3,Audi,A1,Sedans
4,Audi,A1,Sedans
...,...,...,...
15914,Renault,Espace,Van
15915,Renault,Espace,Van
15916,Renault,Espace,Van
15917,Renault,Espace,Van


In [14]:
data[~data["body_type"].notna()]

,make,model,body_type
3175,Audi,A3,None
3255,Audi,A3,None
3975,Audi,A3,None
3997,Audi,A3,None
4206,Audi,A3,None
4297,Audi,A3,None
4298,Audi,A3,None
5718,Opel,Astra,None
5938,Opel,Astra,None
5940,Opel,Astra,None


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   make       15919 non-null  object
 1   model      15919 non-null  object
 2   body_type  15859 non-null  object
dtypes: object(3)
memory usage: 373.2+ KB


### Short_description inceleyelim.

In [16]:
temp = df["short_description"].copy() 

In [17]:
temp.head(50)

0            Sportback 1.4 TDI S-tronic Xenon Navi Klima
1                                         1.8 TFSI sport
2     Sportback 1.6 TDI S tronic Einparkhilfe plus+music
3                                1.4 TDi Design S tronic
4     Sportback 1.4 TDI S-Tronic S-Line Ext. admired, Xe
5            1.6 TDI Sport DSG *SHZ*Xenon*Navi*Bose*PDC*
6          Sportback 1.6 TDI S-TRONIC S-LINE*NAVI|XENON*
7                     Sportback 1.4 TDI admired S tronic
8                 SPB 1.6 TDI S-tronic Metal plus + Xeno
9                  SPORTBACK TFSI ULTRA 95 S-TRONIC AMB.
10           SPORTBACK1.6 TDI 116 CV S tronic Metal plus
11    Sportback Sport »1.4 TFSI|Xenon|Navi|EPH+|SHZ|S tr
12          Sportback 1.4 TFSI S-tronic *S line*NAV*XEN*
13           1.4 TFSI 150ch COD Ambition Luxe S tronic 7
14     Sportback 1.0 TFSI S-TRonic*Navi*Xenon*APS* Sport
15                     SPB 1.6 TDI 116 CV S tronic Sport
16    SPB 1.6 TDI 116 CV Design S-Line SPB 1.6 TDI 116 C
17            1.4 TDi S tronic*

In [18]:
data["cc"] = temp.str.extract("(\d\.\d)").astype("float")  # bu şekilde alınca 5068 tane eksik değer var.

In [19]:
data["cc"].value_counts(dropna=False)

NaN    5068
1.6    3891
1.4    2535
1.0    1334
1.2     957
1.5     890
2.0     888
1.3     135
1.8      60
0.9      49
4.0      28
2.5      21
4.3       9
5.7       8
5.5       3
1.7       3
5.0       3
1.1       3
0.8       2
3.0       2
6.0       2
9.8       2
3.9       2
6.1       1
5.1       1
7.8       1
5.6       1
8.9       1
0.2       1
4.2       1
2.2       1
9.6       1
5.3       1
8.5       1
0.7       1
7.9       1
0.0       1
4.6       1
0.6       1
9.9       1
7.3       1
8.8       1
8.4       1
2.8       1
4.5       1
2.3       1
0.3       1
Name: cc, dtype: int64

In [20]:
data[~data["cc"].notna()]

,make,model,body_type,cc
9,Audi,A1,Sedans,NaN
29,Audi,A1,Compact,NaN
48,Audi,A1,Compact,NaN
55,Audi,A1,Compact,NaN
63,Audi,A1,Sedans,NaN
...,...,...,...,...
15914,Renault,Espace,Van,NaN
15915,Renault,Espace,Van,NaN
15916,Renault,Espace,Van,NaN
15917,Renault,Espace,Van,NaN


### Price inceleyelim.

In [21]:
temp = df["price"].copy()

In [22]:
data["price"]=temp

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   make       15919 non-null  object 
 1   model      15919 non-null  object 
 2   body_type  15859 non-null  object 
 3   cc         10851 non-null  float64
 4   price      15919 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 622.0+ KB


### vat inceleyelim.

In [24]:
temp = df["vat"].copy()   # ???? autoscout24 sitesinde vat deductible seçmeden 1.6 milyon araba varken bunu seçince 600 bine düşüyor.

In [25]:
data["vat"] = temp

In [26]:
data["vat"].value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

### km inceleyelim.

In [27]:
temp = df["km"].copy() 

In [28]:
temp

0        56,013 km
1        80,000 km
2        83,450 km
3        73,000 km
4        16,200 km
           ...    
15914         - km
15915     9,900 km
15916        15 km
15917        10 km
15918         - km
Name: km, Length: 15919, dtype: object

In [29]:
data["km"] = temp.str.replace(r'\D+', '', regex=True)

In [30]:
data[data["km"]==""]   # 1024 tane "" değer var.

,make,model,body_type,cc,price,vat,km
743,Audi,A1,Sedans,NaN,25900,None,
869,Audi,A1,Sedans,1.0,21300,VAT deductible,
946,Audi,A1,Compact,NaN,21406,None,
977,Audi,A1,Compact,1.0,21200,VAT deductible,
980,Audi,A1,Compact,NaN,21100,None,
...,...,...,...,...,...,...,...
15890,Renault,Espace,Station wagon,NaN,42490,VAT deductible,
15902,Renault,Espace,Sedans,NaN,41043,VAT deductible,
15912,Renault,Espace,Van,NaN,39950,VAT deductible,
15914,Renault,Espace,Van,NaN,39950,VAT deductible,


In [31]:
data.km.value_counts(dropna=False)

10        1045
          1024
1          367
5          170
50         148
          ... 
35152        1
93307        1
178000       1
25902        1
231879       1
Name: km, Length: 6690, dtype: int64

### registration inceleyelim. diğer bir sütun olan "first_registration" ile aynı veriye sahip olduğundan duplicate oluşuyor ve birini df dahil etmeyeceğiz.

In [32]:
temp = df["registration"].copy()    # 1597 tane eksik değer var.

In [33]:
temp

0        01/2016
1        03/2017
2        02/2016
3        08/2016
4        05/2016
          ...   
15914        -/-
15915    01/2019
15916    03/2019
15917    06/2019
15918    01/2019
Name: registration, Length: 15919, dtype: object

In [34]:
data["registration"] = temp.str.extract('(\d{4})')

In [35]:
data["registration"] = data["registration"].astype("float")

In [36]:
data["registration"].value_counts(dropna=False)   

2018.0    4522
2016.0    3674
2017.0    3273
2019.0    2853
NaN       1597
Name: registration, dtype: int64

### prev_owner inceleyelim. Aşağıdaki "previos_owner" ile duplicate olacağından biri df dahil edilmeyecek.

In [37]:
temp = df["prev_owner"].copy()    # 6828 tane eksik veri var. Aşağıdaki previous_owners bundan farklı olarak 0 değerler var.

In [38]:
temp.value_counts(dropna=False)

1 previous owner     8294
NaN                  6828
2 previous owners     778
3 previous owners      17
4 previous owners       2
Name: prev_owner, dtype: int64

In [39]:
data["prev_owner"] = temp.str.extract("(\d+)")

In [40]:
data["prev_owner"] = data["prev_owner"].astype("float")

In [41]:
data["prev_owner"].value_counts(dropna=False)   # 6828 tane eksik veri var.

1.0    8294
NaN    6828
2.0     778
3.0      17
4.0       2
Name: prev_owner, dtype: int64

### hp inceleyelim.

In [42]:
temp = df["hp"].copy()

In [43]:
temp.value_counts(dropna=False)  # 88 eksik var.(-)

85 kW     2542
66 kW     2122
81 kW     1402
100 kW    1308
110 kW    1112
70 kW      888
125 kW     707
51 kW      695
55 kW      569
118 kW     516
92 kW      466
121 kW     392
147 kW     380
77 kW      345
56 kW      286
54 kW      276
103 kW     253
87 kW      232
165 kW     194
88 kW      177
60 kW      160
162 kW      98
- kW        88
74 kW       81
96 kW       72
71 kW       59
101 kW      47
67 kW       40
154 kW      39
122 kW      35
119 kW      30
164 kW      27
135 kW      24
52 kW       22
82 kW       22
1 kW        20
78 kW       20
146 kW      18
294 kW      18
141 kW      16
57 kW       10
104 kW       8
120 kW       8
191 kW       7
112 kW       7
155 kW       6
117 kW       6
184 kW       5
90 kW        4
65 kW        4
76 kW        4
168 kW       3
149 kW       3
80 kW        3
98 kW        3
93 kW        3
127 kW       2
270 kW       2
53 kW        2
167 kW       2
40 kW        2
63 kW        2
89 kW        2
228 kW       2
143 kW       2
86 kW        2
150 kW    

In [44]:
data["power_hp"] = temp.str.extract("(\d+)")

In [45]:
data["power_hp"] = data["power_hp"].astype("float")

In [46]:
data["power_hp"].value_counts(dropna=False)

85.0     2542
66.0     2122
81.0     1402
100.0    1308
110.0    1112
70.0      888
125.0     707
51.0      695
55.0      569
118.0     516
92.0      466
121.0     392
147.0     380
77.0      345
56.0      286
54.0      276
103.0     253
87.0      232
165.0     194
88.0      177
60.0      160
162.0      98
NaN        88
74.0       81
96.0       72
71.0       59
101.0      47
67.0       40
154.0      39
122.0      35
119.0      30
164.0      27
135.0      24
82.0       22
52.0       22
1.0        20
78.0       20
146.0      18
294.0      18
141.0      16
57.0       10
120.0       8
104.0       8
191.0       7
112.0       7
155.0       6
117.0       6
184.0       5
90.0        4
76.0        4
65.0        4
149.0       3
98.0        3
93.0        3
80.0        3
168.0       3
150.0       2
63.0        2
140.0       2
86.0        2
89.0        2
40.0        2
167.0       2
53.0        2
228.0       2
127.0       2
143.0       2
270.0       2
9.0         1
44.0        1
123.0       1
195.0 

### Type inceleyelim.

In [47]:
temp = df["Type"].copy()   # buradan vehicle condition ve fuel_type çıkardık.

In [48]:
temp.apply(pd.Series)[1]

0                  Used
1                  Used
2                  Used
3                  Used
4                  Used
              ...      
15914               New
15915              Used
15916    Pre-registered
15917    Pre-registered
15918     Demonstration
Name: 1, Length: 15919, dtype: object

In [49]:
data["vehicle_condition"] = temp.apply(pd.Series)[1]

In [50]:
data["vehicle_condition"].value_counts(dropna=False)  # 2 tane eksik değer var.

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: vehicle_condition, dtype: int64

In [51]:
data[~data["vehicle_condition"].notna()]

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition
2765,Audi,A3,Sedans,2.0,17900,None,115137,2016.0,NaN,110.0,NaN
5237,Audi,A3,Sedans,1.6,25400,None,,NaN,NaN,85.0,NaN


In [52]:
temp.apply(pd.Series)[3]  # 2 tane veri eksik

0                          Diesel (Particulate Filter)
1                                             Gasoline
2                          Diesel (Particulate Filter)
3                          Diesel (Particulate Filter)
4                          Diesel (Particulate Filter)
                             ...                      
15914                      Diesel (Particulate Filter)
15915    Super 95 / Super Plus 98 (Particulate Filter)
15916                                           Diesel
15917                                           Diesel
15918                                         Super 95
Name: 3, Length: 15919, dtype: object

In [45]:
data["fuel_1"] = temp.apply(pd.Series)[3]

KeyError: 3

In [ ]:
data["fuel_1"].value_counts(dropna=False)

In [46]:
data

,make,model,body_type,cc,price,vat,km,registration,prev_owner
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0
...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0
15916,Renault,Espace,Van,NaN,39875,VAT deductible,15,2019.0,1.0
15917,Renault,Espace,Van,NaN,39700,VAT deductible,10,2019.0,NaN


### Previos Owner inceleyelim.

In [53]:
temp = df["Previous Owners"].copy()   # burada yukarıdaki prev_owner dan farklı olarak 0 olan değerler var. Bunlara göre bu sütunlardan birini düşürüceğiz.

In [54]:
data["previous_owners"]= temp.apply(pd.Series)[0].str.extract("(\d+)")

In [55]:
data["previous_owners"].value_counts(dropna=False)

1      8294
NaN    6640
2       778
0       188
3        17
4         2
Name: previous_owners, dtype: int64

In [56]:
data[data["previous_owners"]=="0"]

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners
47,Audi,A1,Sedans,1.6,11790,None,60000,2016.0,NaN,85.0,Used,0
418,Audi,A1,Sedans,1.6,15900,None,58000,2016.0,NaN,66.0,Used,0
586,Audi,A1,Sedans,1.0,13500,None,50707,2016.0,NaN,70.0,Used,0
648,Audi,A1,Sedans,1.4,12900,None,64000,2016.0,NaN,66.0,Used,0
734,Audi,A1,Sedans,NaN,30000,None,0,NaN,NaN,85.0,New,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15301,Renault,Espace,Van,NaN,37290,None,,NaN,NaN,146.0,New,0
15408,Renault,Espace,Van,NaN,24900,None,91883,2016.0,NaN,118.0,Used,0
15668,Renault,Espace,Van,NaN,39290,None,0,NaN,NaN,118.0,New,0
15853,Renault,Espace,Van,2.0,43950,VAT deductible,100,NaN,NaN,147.0,New,0


### Next_Inspection inceleyelim. Eksik veri sayısı % 78 olduğundan dolayı kullanmadık.

In [57]:
temp = df["Next Inspection"].copy()  # bu sütün silinebilir. Eksik veri sayısı 12384. Ayrıca araba alırken önemli hususlar arasında yok.

In [58]:
data["Next_Inspection"]= temp.apply(pd.Series)[0].str.extract('(\d{4})')

In [59]:
data["Next_Inspection"].value_counts(dropna=False)

NaN     12384
2021     1601
2020      694
2022      688
2019      438
2023       47
2018       38
2017       13
2016        6
2001        5
1955        1
1999        1
1921        1
2014        1
2024        1
Name: Next_Inspection, dtype: int64

In [60]:
data.drop("Next_Inspection", axis=1, inplace=True)

### Inspection_new inceleyelim.

In [61]:
temp = df["Inspection new"].copy()     # bu sütün silinebilir veya boş olanları bakılıp bunlar No ile doldurulabilir. Eksik veri sayısı 11987 veya boş olan değerlere No yazılabilir.

In [62]:
temp.apply(pd.Series)[0].str.extract('(\w+)')

,0
0,Yes
1,NaN
2,NaN
3,NaN
4,Yes
...,...
15914,NaN
15915,NaN
15916,Yes
15917,NaN


In [64]:
data["Inspection_new"] = temp.apply(pd.Series)[0].str.extract('(\w+)')

In [65]:
data["Inspection_new"].value_counts(dropna=False)

NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64

In [66]:
data[~data["Inspection_new"].notna()]

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN
5,Audi,A1,Sedans,1.6,15090,None,63668,2016.0,1.0,85.0,Used,1,NaN
8,Audi,A1,Sedans,1.6,16700,None,57000,2016.0,1.0,85.0,Used,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15912,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN
15913,Renault,Espace,Van,NaN,39950,VAT deductible,1000,2019.0,NaN,165.0,Demonstration,NaN,NaN
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN


### Warranty inceleyelim.

In [59]:
temp = df["Warranty"].copy()   #11066 eksik veri silinebilir veya boş olanlar için guarantee yok denilebilir.

In [60]:
temp.apply(pd.Series)[0].str.extract('(\d{1,3})')

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
15914,24
15915,NaN
15916,NaN
15917,NaN


In [61]:
data["guarante"] = temp.apply(pd.Series)[0].str.extract('(\d{1,3})')

In [62]:
data["guarante"].value_counts(dropna=False)

NaN    11066
12      2594
24      1118
60       401
36       279
48       149
6        125
72        59
3         33
23        11
18        10
20         7
25         6
2          5
50         4
16         4
26         4
34         3
13         3
4          3
19         3
1          3
11         2
14         2
9          2
46         2
28         2
45         2
17         2
22         2
21         2
30         1
7          1
65         1
10         1
33         1
8          1
49         1
15         1
40         1
47         1
56         1
Name: guarante, dtype: int64

### Full Service inceleyelim.

In [63]:
temp = df["Full Service"].copy()   # bunu düşebiliriz. Kullanılacak veri yok. 

In [64]:
temp.apply(pd.Series)[0].str.extract("(\w+)")

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
15914,NaN
15915,NaN
15916,NaN
15917,NaN


### Non-smoking Vehicle inceleyelim.

In [65]:
temp = df["Non-smoking Vehicle"].copy()  # düşülebilir. Veri yok.

In [66]:
temp.apply(pd.Series)[0]

0         \n
1        NaN
2        NaN
3         \n
4         \n
        ... 
15914    NaN
15915     \n
15916     \n
15917    NaN
15918    NaN
Name: 0, Length: 15919, dtype: object

### Null inceleyelim. İçinde değer olmadığından dolayı düşülecek.

### Make inceleyelim. Yukarıda çektiğimiz için düşülecek.

### Model inceleyelim. Yukarıda çektiğimiz için düşülecek.

### offer_number inceleyelim. Düşülecek. Kullanım yeri olmayan gereksiz bir bilgi. 

### First Registration inceleyelim. Ancak yukarıda çektiğimiz registration ile aynı veriye sahip olduğundan dolayı duplicate olacak ve bunlardan birinin düşülmesi gerekir.

In [67]:
temp = df['First Registration'].copy()

In [68]:
temp.apply(pd.Series)[1].str.extract('(\d{4})')

,0
0,2016
1,2017
2,2016
3,2016
4,2016
...,...
15914,NaN
15915,2019
15916,2019
15917,2019


In [69]:
data["first_registration"] = temp.apply(pd.Series)[1].str.extract('(\d{4})')

In [70]:
data["first_registration"].value_counts(dropna=False)  # 1597 eksik veri var. Yukarıdaki registration ile aynı veriye sahip olduğundan dolayı birini kullanmamız yeterli.

2018    4522
2016    3674
2017    3273
2019    2853
NaN     1597
Name: first_registration, dtype: int64

In [71]:
data[~data["first_registration"].notna()][["first_registration","registration"]]

,first_registration,registration
122,NaN,NaN
710,NaN,NaN
734,NaN,NaN
741,NaN,NaN
743,NaN,NaN
...,...,...
15896,NaN,NaN
15902,NaN,NaN
15907,NaN,NaN
15912,NaN,NaN


### Body Color inceleyelim.

In [67]:
temp = df['Body Color'].copy()

In [68]:
temp.apply(pd.Series)[1].str.extract('(\w+)')

,0
0,Black
1,Red
2,Black
3,Brown
4,Black
...,...
15914,Grey
15915,Grey
15916,White
15917,Grey


In [69]:
data["body_color"] = temp.apply(pd.Series)[1].str.extract('(\w+)')

In [70]:
data["body_color"].value_counts(dropna=False)   # 597 eksik veri var. Bunlardan 1000 üzerin olanlar kalabilir diğerlerini other olarak atayabiliriz.

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: body_color, dtype: int64

### Paint Type inceleyelim.

In [71]:
temp = df['Paint Type'].copy()

In [72]:
temp.apply(pd.Series)[0].str.extract("(\w+)")   

,0
0,Metallic
1,NaN
2,Metallic
3,Metallic
4,Metallic
...,...
15914,Metallic
15915,Metallic
15916,NaN
15917,NaN


In [73]:
data["paintwork"] = temp.apply(pd.Series)[0].str.extract("(\w+.\w+)")

In [74]:
data["paintwork"].value_counts(dropna=False)   # 5772 eksik veri var. Bunları metallic olanlar ve olmayanlar diye 2 gruba ayırabiliriz.

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: paintwork, dtype: int64

In [67]:
data.head(3)

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic


### Body Color Original inceleyelim.

In [75]:
temp = df['Body Color Original'].copy()   # 3759 eksik veri var. Bu sütunun dolu olanları ile eksik veri olanların fiyata etkisine baktığımız zaman dikkate değer bir fark olduğu görüldü.

In [76]:
temp.apply(pd.Series)[0].str.extract("(\w+.\w+)")

,0
0,Mythosschwarz
1,NaN
2,mythosschwarz metallic
3,NaN
4,Mythosschwarz Metallic
...,...
15914,Grigio scuro
15915,Stahl-Grau
15916,arktis-weiß
15917,Grigio


In [77]:
data['body_color_original'] = temp.apply(pd.Series)[0].str.extract("(\w+.\w+)")

In [78]:
data['body_color_original'].value_counts(dropna=False)

NaN                    3759
Onyx Schwarz            389
Bianco                  282
Argon Silber            273
Quarz Grau              266
                       ... 
Scuba Blauw               1
Blauw metallic            1
CORTINAWIT                1
Utopiablau metallic       1
Cassio-Grau               1
Name: body_color_original, Length: 1416, dtype: int64

In [75]:
data[~data['body_color_original'].notna()].groupby(["make","model"])["price"].mean()

make     model   
Audi     A1          16852.407692
         A3          20820.432028
Opel     Astra       13327.302966
         Corsa        9423.137500
         Insignia    17925.170923
Renault  Clio        11259.698020
         Duster      13230.176471
         Espace      24874.170370
Name: price, dtype: float64

In [76]:
data[data['body_color_original'].notna()].groupby(["make","model"])["price"].mean()

make     model   
Audi     A1          19364.395893
         A2          28200.000000
         A3          21065.331539
Opel     Astra       16410.844206
         Corsa       11495.341576
         Insignia    22325.579703
Renault  Clio        12274.836172
         Espace      32029.769764
Name: price, dtype: float64

### Upholstery inceleyelim.

In [79]:
temp = df['Upholstery'].copy()

In [80]:
temp

0               [\nCloth, Black\n]
1                [\nCloth, Grey\n]
2               [\nCloth, Black\n]
3                              NaN
4               [\nCloth, Black\n]
                   ...            
15914                          NaN
15915                  [\nCloth\n]
15916    [\nFull leather, Black\n]
15917           [\nPart leather\n]
15918    [\nFull leather, Brown\n]
Name: Upholstery, Length: 15919, dtype: object

In [81]:
data["upholstery"] = temp.apply(pd.Series)[0].str.split(",", n = 1, expand = True)[0].str.extract("(\w+..\w+)")

In [82]:
data["upholstery"].value_counts(dropna=False)  # 3720 eksik veri. 

# Alcantara is velvety but non-marking fabric when rubbed. 

# Burada ayrıca cloth, Part leather, Full Leather, Velour, Alcantara ve other dışındakiler color olup bunları interior_color atabiliriz.

Cloth           8423
NaN             3720
Part leather    1499
Full leather    1009
Black            491
Other            368
Grey             273
Velour            60
alcantara         57
Brown             12
Beige              3
Blue               2
White              2
Name: upholstery, dtype: int64

In [83]:
data["interior_color"] = temp.apply(pd.Series)[0].str.split(",", n = 1, expand = True)[1].str.extract("(\w+..\w+)")

In [84]:
data["interior_color"].value_counts(dropna=False)  # 5870 eksik veri var.

Black     7710
NaN       5870
Grey      1103
Other      960
Brown      195
Beige       51
Blue        14
White       11
Yellow       4
Orange       1
Name: interior_color, dtype: int64

### Body inceleyelim. Yukarıda body_type ile duplicate olduğundan dolayı düşülecek.

In [93]:
temp = df['Body']

In [94]:
temp.apply(pd.Series)[1] #.str.extract("(\w+.\w+)").value_counts()

0        Sedans
1        Sedans
2        Sedans
3        Sedans
4        Sedans
          ...  
15914       Van
15915       Van
15916       Van
15917       Van
15918       Van
Name: 1, Length: 15919, dtype: object

In [95]:
data["body"] = temp.apply(pd.Series)[1]

In [96]:
data["body"].value_counts(dropna=False)  # 60 eksik veri var. Ancak yukarıdaki body_type ile aynı olduğundan dolayı düşülecek.

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body, dtype: int64

### Nr. of Doors inceleyelim.

In [85]:
temp = df['Nr. of Doors'].copy()

In [86]:
data['door_nr'] = temp.apply(pd.Series)[0].str.extract("(\d+)")

In [87]:
data['door_nr'].value_counts(dropna=False)  # 212 eksik veri var.

5      11575
4       3079
3        832
2        219
NaN      212
1          1
7          1
Name: door_nr, dtype: int64

### Nr. of Seats inceleyelim.

In [88]:
temp = df['Nr. of Seats'].copy()

In [89]:
data['seat_nr'] = temp.apply(pd.Series)[0].str.extract("(\d+)")

In [90]:
data['seat_nr'].value_counts(dropna=False)  # 977 eksik veri var.

5      13336
4       1125
NaN      977
7        362
2        116
6          2
3          1
Name: seat_nr, dtype: int64

### Model Code inceleyelim. Bu eksik verilerin tamamlanmasında kullanılabilir. Bu model kodu olanlarda bilgilerin aynı olması gerekir.

In [91]:
temp = df['Model Code'].copy()

In [92]:
data["model_code"] = temp.apply(pd.Series)[0].str.extract("(\S+)")

In [93]:
data["model_code"].value_counts(dropna=False)   # 10941 eksik veri var.

NaN         10941
0035/BCB      268
0588/BNO      245
0588/BDB      206
0588/BHX      188
            ...  
0035/BGI        1
0588/AZZ        1
1844/AEG        1
1844/AEN        1
0035/AVN        1
Name: model_code, Length: 233, dtype: int64

In [92]:
data[data["model_code"].notna()]  #["model_code"] =="0035/BCB"

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF
7,Audi,A1,Compact,1.4,14480,VAT deductible,14986,2016.0,1.0,66.0,Used,1,Yes,Red,Metallic,Misanorot Perleffekt,Cloth,Black,5,5,0588/BDF
11,Audi,A1,Sedans,1.4,17990,None,26415,2016.0,1.0,92.0,Used,1,Yes,Silver,Uni/basic,Florettsilber Metallic,Cloth,Black,4,NaN,0588/BDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15907,Renault,Espace,Van,1.8,39990,VAT deductible,50,NaN,NaN,165.0,New,NaN,NaN,Grey,Metallic,titaniumgraumetallic,Full leather,NaN,5,7,3333/BHJ
15910,Renault,Espace,Van,NaN,39980,VAT deductible,8500,2019.0,1.0,165.0,Demonstration,1,NaN,Blue,Metallic,Kosmos-Blau,Full leather,Other,5,5,3333/BHJ
15911,Renault,Espace,Van,NaN,39980,VAT deductible,100,2019.0,1.0,118.0,Pre-registered,1,NaN,White,Metallic,Perlmutt-Weiß,Full leather,Grey,5,7,3333/BDQ
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000


In [107]:
df2[df2["model_code"] =="0035/BCB"

SyntaxError: unexpected EOF while parsing (<ipython-input-107-f29486aa047c>, line 1)

### Gearing Type inceleyelim.

In [94]:
temp = df['Gearing Type'].copy()

In [95]:
data["gear"] = temp.apply(pd.Series)[1]

In [96]:
data["gear"].value_counts(dropna=False)   # veri tam. 

Manual            8153
Automatic         7297
Semi-automatic     469
Name: gear, dtype: int64

### Displacement inceleyelim. Bu husus araba fiyatına etki eden bir husus olmadığından dolayı düşülebilir.

In [97]:
temp = df["Displacement"].copy()

In [98]:
data["displacement"] = temp.apply(pd.Series)[0].str.extract("(\d+.\d+)")

In [99]:
data["displacement"].value_counts(dropna=False)  # 535 eksik veri var. Ancak bazı düzeltilmesi gereken veriler var.

1,598     4761
999       2438
1,398     1314
1,399      749
1,229      677
1,956      670
1,461      595
1,490      559
NaN        535
1,422      467
1,197      353
898        351
1,395      320
1,968      301
1,149      288
1,618      212
1,798      210
1,498      196
1,600      130
1,248      110
1,997      103
1,364      102
1,400       90
998         72
1,500       50
2,000       46
1,000       40
1,998       25
2,480       20
1,984       18
1,200       18
899         11
1,397       11
160          6
929          5
1,499        5
1,596        4
139          4
900          4
997          4
1,396        3
1,199        3
1,599        3
1,589        2
995          2
1,495        2
1,300        2
16,000       1
140          1
1,390        1
1,369        1
1,584        1
1,100        1
973          1
996          1
1,533        1
890          1
1,568        1
1,580        1
1,350        1
1,368        1
1,696        1
1,856        1
1,195        1
122          1
1,800        1
1,496     

In [99]:
data

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422"
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798"
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598"
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422"
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997"
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798"
15916,Renault,Espace,Van,NaN,39875,VAT deductible,15,2019.0,1.0,146.0,Pre-registered,1,Yes,White,NaN,arktis-weiß,Full leather,Black,5,7,NaN,Automatic,"1,997"
15917,Renault,Espace,Van,NaN,39700,VAT deductible,10,2019.0,NaN,147.0,Pre-registered,NaN,NaN,Grey,NaN,Grigio,Part leather,NaN,5,7,NaN,Automatic,"1,997"


### Cylinders inceleyelim.

In [100]:
temp = df['Cylinders'].copy()

In [101]:
data['cylinders'] = temp.apply(pd.Series)[0].str.extract("(\d+)")

In [102]:
data['cylinders'].value_counts(dropna=False)   # 5680 eksik veri var. Bu sütun silinecek ancak eksik veri doldururken kullanılabilir.

4      8105
NaN    5680
3      2104
5        22
6         3
8         2
2         2
1         1
Name: cylinders, dtype: int64

### Weight inceleyelim.

In [103]:
temp = df['Weight'].copy()

In [104]:
data['weight'] = temp.apply(pd.Series)[0].str.extract("(\d+.\d+)")

In [105]:
data['weight'].value_counts(dropna=False)   # 6976 eksik veri var. Bu sütun silinecek ancak eksik veri doldururken kullanılabilir.

NaN      6976
1,163     574
1,360     356
1,165     301
1,335     242
         ... 
1,277       1
1,589       1
1,252       1
1,891       1
1,598       1
Name: weight, Length: 434, dtype: int64

### Drive chain inceleyelim.

In [121]:
temp = df['Drive chain'].copy()

In [122]:
temp

0        [\nfront\n]
1        [\nfront\n]
2        [\nfront\n]
3                NaN
4        [\nfront\n]
            ...     
15914    [\nfront\n]
15915    [\nfront\n]
15916    [\nfront\n]
15917    [\nfront\n]
15918      [\n4WD\n]
Name: Drive chain, Length: 15919, dtype: object

In [123]:
data["drive_chain"] = temp.apply(pd.Series)[0].str.extract("(\S+)")

In [124]:
data["drive_chain"].value_counts(dropna=False)  # 6858 eksik veri var. Bu sütun silinecek ancak eksik veri doldururken kullanılabilir.

front    8886
NaN      6858
4WD       171
rear        4
Name: drive_chain, dtype: int64

### Fuel inceleyelim. Ancak bu veri yukarıdaki yukarıdan vehicle_condition dan elde edilen fuel_1 ile aynı olabilir. Bu durumda duplicate olacağından dolayı silinecek.

In [106]:
temp = df["Fuel"].copy()  # eksik veri yok. Ancak buradaki veriyi iki gruba ayıracağız. Bunun için func. yazalım.

In [107]:
data["fuel"] = temp.apply(pd.Series)[1].str.extract("(\w+)")

In [108]:
data["fuel"].value_counts(dropna=False)

Diesel      7299
Super       4790
Gasoline    3252
Regular      503
CNG           36
LPG           16
Liquid        10
Others         6
Electric       5
Biogas         1
Domestic       1
Name: fuel, dtype: int64

In [109]:
benzine = ["Gasoline","Super","Super 95","Regular", "Super E10 95","Super Plus 98","Super Plus E10 98","Others"]

lpg = ["LPG","Liquid petroleum gas","CNG","Biogas","Domestic gas H"]


def fueltype(x):
    if x in benzine:
        return "benzine"
    elif x in lpg:
        return "LPG/CNG"
    else :
        return x

data["fuel"] = data.fuel.apply(fueltype)

data["fuel"]

0         Diesel
1        benzine
2         Diesel
3         Diesel
4         Diesel
          ...   
15914     Diesel
15915    benzine
15916     Diesel
15917     Diesel
15918    benzine
Name: fuel, Length: 15919, dtype: object

In [110]:
data["fuel"].value_counts(dropna=False)

benzine     8551
Diesel      7299
LPG/CNG       53
Liquid        10
Electric       5
Domestic       1
Name: fuel, dtype: int64

In [179]:
# df["Fuel"] = df["Fuel"].str[1].str.split("/").str[0].str.strip()

# df["Fuel"] = df.Fuel.str.split("(").str[0].str.strip()

### Consumption inceleyelim. Burada nester list yapısı söz konusu olduğundan önce tüm sütunu bir dataframe atıyoruz. Sütunları yeniden isimlendiriyoruz. Sonra her bir sütunun satırlarındaki list içinden değerleri çıkarıyoruz. Ancak verinin içinde 850 satırda bir kayma durumu söz konusu olduğunu görüyoruz. Bunlar ile ilgili düzeltmeyi yaptıktan sonra ihtiyacımız olan consumption değerlerini çıkararak ortak, city ve country olarak 3 tane sütun elde ediyoruz.

In [111]:
df['Consumption'].value_counts(dropna=False)

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


NaN                                                                                   1906
[[3.9 l/100 km (comb)], [4.1 l/100 km (city)], [3.7 l/100 km (country)]]               304
[[4.2 l/100 km (comb)], [5 l/100 km (city)], [3.7 l/100 km (country)]]                 276
[[5.4 l/100 km (comb)], [6.8 l/100 km (city)], [4.5 l/100 km (country)]]               257
[[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]               253
                                                                                      ... 
[[3.1 l/100 km (comb)], [], [3.5 l/100 km (country)]]                                    1
[[5.2 l/100 km (comb)], [6 l/100 km (city)], [4.8 l/100 km (country)]]                   1
[[5.3 l/100 km (comb)], [6 l/100 km (city)], [5 l/100 km (country)]]                     1
[\n, 6.5 l/100 km (comb), \n, 8.1 l/100 km (city), \n, 5.5 l/100 km (country), \n]       1
[[50 l/100 km (comb)], [64 l/100 km (city)], [42 l/100 km (country)]]                    1

In [112]:
temp = df['Consumption'].copy()

In [113]:
df3 = temp.apply(pd.Series)

In [114]:
df3

,0,1,2,3,4,5,6
0,[3.8 l/100 km (comb)],[4.3 l/100 km (city)],[3.5 l/100 km (country)],NaN,NaN,NaN,NaN
1,[5.6 l/100 km (comb)],[7.1 l/100 km (city)],[4.7 l/100 km (country)],NaN,NaN,NaN,NaN
2,[3.8 l/100 km (comb)],[4.4 l/100 km (city)],[3.4 l/100 km (country)],NaN,NaN,NaN,NaN
3,[3.8 l/100 km (comb)],[4.3 l/100 km (city)],[3.5 l/100 km (country)],NaN,NaN,NaN,NaN
4,[4.1 l/100 km (comb)],[4.6 l/100 km (city)],[3.8 l/100 km (country)],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15914,[5.3 l/100 km (comb)],[6.2 l/100 km (city)],[4.7 l/100 km (country)],NaN,NaN,NaN,NaN
15915,\n,7.4 l/100 km (comb),\n,9.2 l/100 km (city),\n,6.3 l/100 km (country),\n
15916,[5.3 l/100 km (comb)],[6.2 l/100 km (city)],[4.7 l/100 km (country)],NaN,NaN,NaN,NaN
15917,[5.3 l/100 km (comb)],[6.2 l/100 km (city)],[4.7 l/100 km (country)],NaN,NaN,NaN,NaN


In [115]:
df3.rename(columns={0:"comb",1:"city",2:"country",3:"ccity",4:"free",5:"ccountry",6:"free2"}, inplace=True)

In [116]:
df3.comb = df3.comb.apply(pd.Series)

In [117]:
df3.city = df3.city.apply(pd.Series)

In [118]:
df3.country = df3.country.apply(pd.Series)

In [119]:
df3.ccity = df3.ccity.apply(pd.Series)

In [120]:
df3.ccountry = df3.ccountry.apply(pd.Series)

In [121]:
df3 = df3.replace("\n", np.nan)

In [122]:
df3

,comb,city,country,ccity,free,ccountry,free2
0,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
1,5.6 l/100 km (comb),7.1 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
2,3.8 l/100 km (comb),4.4 l/100 km (city),3.4 l/100 km (country),NaN,NaN,NaN,NaN
3,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
4,4.1 l/100 km (comb),4.6 l/100 km (city),3.8 l/100 km (country),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15914,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15915,NaN,7.4 l/100 km (comb),NaN,9.2 l/100 km (city),NaN,6.3 l/100 km (country),NaN
15916,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15917,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN


In [123]:
df3.comb.fillna(df3.city, inplace=True)

In [124]:
df3

,comb,city,country,ccity,free,ccountry,free2
0,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
1,5.6 l/100 km (comb),7.1 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
2,3.8 l/100 km (comb),4.4 l/100 km (city),3.4 l/100 km (country),NaN,NaN,NaN,NaN
3,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
4,4.1 l/100 km (comb),4.6 l/100 km (city),3.8 l/100 km (country),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15914,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15915,7.4 l/100 km (comb),7.4 l/100 km (comb),NaN,9.2 l/100 km (city),NaN,6.3 l/100 km (country),NaN
15916,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15917,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN


In [125]:
df3.country.fillna(df3.ccountry, inplace=True)

In [126]:
df3.city = np.where(df3.city.str.contains("comb"), df3.ccity, df3.city)  


In [127]:
df3

,comb,city,country,ccity,free,ccountry,free2
0,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
1,5.6 l/100 km (comb),7.1 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
2,3.8 l/100 km (comb),4.4 l/100 km (city),3.4 l/100 km (country),NaN,NaN,NaN,NaN
3,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country),NaN,NaN,NaN,NaN
4,4.1 l/100 km (comb),4.6 l/100 km (city),3.8 l/100 km (country),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15914,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15915,7.4 l/100 km (comb),9.2 l/100 km (city),6.3 l/100 km (country),9.2 l/100 km (city),NaN,6.3 l/100 km (country),NaN
15916,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN
15917,5.3 l/100 km (comb),6.2 l/100 km (city),4.7 l/100 km (country),NaN,NaN,NaN,NaN


In [128]:
data["consumption_comb"] = df3.comb.str.extract("(\d\d|\d.\d|\d)")

In [129]:
data["consumption_city"] = df3.city.str.extract("(\d\d|\d.\d|\d)")

In [130]:
data["consumption_country"] = df3.country.str.extract("(\d\d|\d.\d|\d)")

In [131]:
data["consumption_comb"].value_counts(dropna=False)

NaN    1925
5.4     770
3.9     733
4       714
5.1     657
4.4     623
5.6     618
4.7     602
3.8     585
5       551
4.8     546
4.5     526
5.2     455
4.6     445
4.2     435
4.9     393
5.3     380
5.5     380
5.9     369
3.7     369
5.7     342
4.1     342
6       331
3.3     326
4.3     307
3.5     307
6.2     216
3.6     194
6.3     181
6.1     175
5.8     164
6.6     148
3.4     143
6.8     136
6.4      75
3        69
7.4      66
6.7      45
6.5      43
7.1      39
10       34
6.9      27
3.2      25
8.3      20
7.6      14
7        10
3.1       7
7.8       6
7.2       6
8         5
51        4
8.6       4
7.9       3
8.7       3
1.6       3
7.3       2
0         2
38        2
8.1       2
40        2
9.1       1
7.5       1
54        1
1.2       1
32        1
55        1
43        1
46        1
50        1
11        1
1         1
13        1
33        1
Name: consumption_comb, dtype: int64

In [191]:
data

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,NaN,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill H...","Bluetooth,Hands-free equipment,On-board computer,Radio","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog..."
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,NaN,7.1,4.7,129,Euro 6,"Air conditioning,Automatic climate control,Hill Holder,Leather steering wheel,Lumbar support,Par...","Bluetooth,Hands-free equipment,On-board computer,Radio,Sound system","Alloy wheels,Sport seats,Sport suspension,Voice Control","ABS,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side a..."
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,NaN,4.4,3.4,99,Euro 6,"Air conditioning,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Multi...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Imm..."
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,NaN,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Auxiliary heating,Electrical side mirrors,Heads-up display,Hill Holder,Le...","Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,Sound system,USB","Alloy wheels,Sport seats,Voice Control","ABS,Alarm system,Central door lock with remote control,Driver drowsiness detection,Driver-side a..."
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,NaN,Diesel,NaN,4.6,3.8,109,Euro 6,"Air conditioning,Armrest,Automatic climate control,Electrical side mirrors,Hill Holder,Leather s...","Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB","Alloy wheels,Sport package,Sport suspension,Voice Control","ABS,Central door lock,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Iso..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997",4,"1,758",Diesel,NaN,6.2,4.7,139,NaN,"Air conditioning,Automatic climate control,Cruise control,Electrically adjustable seats,Electric...","Bluetooth,Digital radio,Hands-free equipment,On-board computer,Radio,USB","Alloy wheels,Touch screen","ABS,Central door lock,Central door lock with remote control,Driver-side airbag,Electronic stabil..."
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798",4,"1,708",benzine,7.4,9.2,6.3,168,NaN,"Air conditioning,Automatic climate control,Cruise control,Electrically heated windshield,Electri...","Bluetooth,Digital radio,Hands-free equipment,Radio,USB","Alloy wheels,Touch screen,Voice Control","ABS,Adaptive Cruise Control,Blind spot monitor,Central door lock,Daytime running lights,Driver-s..."
15916,Renault,Espace,Van,NaN,398

In [127]:
data["consumption_country"].value_counts(dropna=False)

NaN    2376
4.2    1178
3.7    1089
4.4     898
4.5     848
3.8     802
3.9     777
4.7     615
4.1     592
4       589
3.5     525
4.3     520
4.6     516
3.6     486
4.9     481
3.1     481
3.3     457
3.4     353
4.8     341
5.3     278
5.4     239
5.1     232
5.7     204
3.2     180
5.6     154
3       142
5       130
5.2     117
6.3      52
6        42
5.5      36
10       33
5.8      30
7.7      20
6.6      19
2.9      16
6.4      15
2.8       9
0         9
7.3       4
6.5       4
44        4
7.1       3
7         3
7.8       2
35        2
37        2
6.9       2
6.7       2
5.9       2
6.1       2
1         1
2         1
8.6       1
42        1
8         1
7.6       1
Name: consumption_country, dtype: int64

### CO2 Emission inceleyelim.

In [132]:
temp = df['CO2 Emission'].copy()   # 2436 eksik değer var.

In [133]:
data["CO2_emission"] = temp.apply(pd.Series)[0].str.extract("(\d+)")

In [134]:
data["CO2_emission"].value_counts(dropna=False)

NaN    2436
120     740
99      545
97      537
104     501
       ... 
990       1
183       1
13        1
180       1
177       1
Name: CO2_emission, Length: 120, dtype: int64

### Emission Class inceleyelim.

In [135]:
temp = df['Emission Class'].copy()

In [136]:
data["emission_class"] = temp.apply(pd.Series)[0].str.extract("(\w.+)")

In [137]:
data["emission_class"].value_counts(dropna=False)   # 3628 eksik değer var. 

Euro 6          10139
NaN              3628
Euro 6d-TEMP     1845
Euro 6c           127
Euro 5             78
Euro 6d            62
Euro 4             40
Name: emission_class, dtype: int64

### Comfort & Convenience inceleyelim. Bu feature ile ilgili olarak araçlarda burada geçen özelliklerin olup olmadığına göre sütunlar oluşturacağız.  

In [138]:
df['\nComfort & Convenience\n'] = [",".join(item) if type(item)==list else item for item in df['\nComfort & Convenience\n']]

data["comfort_convenience"] = df['\nComfort & Convenience\n'].str.extract("(\w.+)").copy()  # 920 eksik veri var.

In [135]:
data["comfort_convenience"].str.get_dummies(sep=",")

,Air conditioning,Air suspension,Armrest,Automatic climate control,Auxiliary heating,Cruise control,Electric Starter,Electric tailgate,Electrical side mirrors,Electrically adjustable seats,Electrically heated windshield,Heads-up display,Heated steering wheel,Hill Holder,Keyless central door lock,Leather seats,Leather steering wheel,Light sensor,Lumbar support,Massage seats,Multi-function steering wheel,Navigation system,Panorama roof,Park Distance Control,Parking assist system camera,Parking assist system self-steering,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Seat ventilation,Split rear seats,Start-stop system,Sunroof,Tinted windows,Wind deflector,Windshield
0,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0
3,0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0
4,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,1,0,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,0,1,0,0
15915,1,0,0,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0
15916,1,0,1,1,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,0,0,0
15917,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [136]:
comfort = data["comfort_convenience"].str.get_dummies(sep=",")

In [147]:
data.join(comfort)  # bunu en son çalıştıralım.

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security,Air conditioning,Air suspension,Armrest,Automatic climate control,Auxiliary heating,Cruise control,Electric Starter,Electric tailgate,Electrical side mirrors,Electrically adjustable seats,Electrically heated windshield,Heads-up display,Heated steering wheel,Hill Holder,Keyless central door lock,Leather seats,Leather steering wheel,Light sensor,Lumbar support,Massage seats,Multi-function steering wheel,Navigation system,Panorama roof,Park Distance Control,Parking assist system camera,Parking assist system self-steering,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Seat ventilation,Split rear seats,Start-stop system,Sunroof,Tinted windows,Wind deflector,Windshield
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,3.8,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...","Alloy wheels,Catalytic Con...","ABS,Central door lock,Dayt...",1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,5.6,7.1,4.7,129,Euro 6,"Air conditioning,Automatic...","Bluetooth,Hands-free equip...","Alloy wheels,Sport seats,S...","ABS,Central door lock,Cent...",1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,3.8,4.4,3.4,99,Euro 6,"Air conditioning,Cruise co...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Dayt...",1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,3.8,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Aux...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport seats,V...","ABS,Alarm system,Central d...",0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,NaN,Diesel,4.1,4.6,3.8,109,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport package...","ABS,Central door lock,Driv...",1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen","ABS,Central door lock,Cent...",1,0,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,0,1,0,0
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798",4,"1,708",benzine,7.4,9.2,6.3,168,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen,...","ABS,Adaptive Cruise 

### 2. yöntem olarak herbir eleman için olanlara 1 olmayanlara 0 atayarak da aynı sonuca ulaşabiliriz.

In [291]:

data["air_conditioning"] = np.where(temp[0].str.contains("Air conditioning"), 1, 0)  

data["air_conditioning"]

0        1
1        1
2        1
3        0
4        1
        ..
15914    1
15915    1
15916    1
15917    1
15918    1
Name: air_conditioning, Length: 15919, dtype: int32

In [292]:
data.head(3)

,make,model,body_type,cc,price,vat,km,registration,prev_owner,previous_owners,Next_Inspection,Inspection_new,guarante,first_registration,body_color,paintwork,body_color_original,upholstery,interior_color,body,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,drive_chain,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,air_conditioning,cruise_control,automatic_climate_control
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,2,2021,Yes,NaN,2016,Black,Metallic,Mythosschwarz,Cloth,Black,Sedans,5,5,0588/BDF,Automatic,"1,422",3,"1,220",front,Diesel (Particulate Filter),3.8,4.3,3.5,99,Euro 6,1,1,1
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,NaN,NaN,NaN,NaN,2017,Red,NaN,NaN,Cloth,Grey,Sedans,3,4,0588/BCY,Automatic,"1,798",4,"1,255",front,Gasoline,5.6,7.1,4.7,129,Euro 6,1,0,1
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,1,NaN,NaN,NaN,2016,Black,Metallic,mythosschwarz metallic,Cloth,Black,Sedans,4,4,NaN,Automatic,"1,598",NaN,NaN,front,Diesel (Particulate Filter),3.8,4.4,3.4,99,Euro 6,1,1,0


In [293]:
data["cruise_control"] = np.where(temp[0].str.contains("Cruise control"), 1, 0)  

data["cruise_control"]

0        1
1        0
2        1
3        0
4        0
        ..
15914    1
15915    1
15916    1
15917    1
15918    1
Name: cruise_control, Length: 15919, dtype: int32

In [294]:
data["automatic_climate_control"] = np.where(temp[0].str.contains("Automatic climate control"), 1, 0)  

data["automatic_climate_control"]

0        1
1        1
2        0
3        0
4        1
        ..
15914    1
15915    1
15916    1
15917    1
15918    1
Name: automatic_climate_control, Length: 15919, dtype: int32

In [295]:
data["armrest"] = np.where(temp[0].str.contains("Armrest"), 1, 0)  

data["armrest"]

0        1
1        0
2        0
3        1
4        1
        ..
15914    0
15915    0
15916    1
15917    0
15918    0
Name: armrest, Length: 15919, dtype: int32

In [ ]:
data["electrical_side_mirrors"] = np.where(temp[0].str.contains("Electrical side mirrors"), 1, 0)  

data["electrical_side_mirrors"]

In [ ]:
data["navigation_system"] = np.where(temp[0].str.contains("Navigation system"), 1, 0)  

data["navigation_system"]

In [ ]:
data["multifunction_steering_wheel"] = np.where(temp[0].str.contains("Multi-function steering wheel"), 1, 0)  

data["multifunction_steering_wheel"]

In [ ]:
data["power_windows"] = np.where(temp[0].str.contains("Power windows"), 1, 0)  

data["power_windows"]

In [ ]:
data["sunroof"] = np.where(temp[0].str.contains("Sunroof"), 1, 0)  

data["sunroof"]

In [ ]:
data["leather_steering_wheel "] = np.where(temp[0].str.contains("Leather steering wheel"), 1, 0)  

data["leather_steering_wheel "]

In [ ]:
data["park_distance_control"] = np.where(temp[0].str.contains("Park Distance Control"), 1, 0)  

data["park_distance_control"]

In [ ]:
data["hill_holder"] = np.where(temp[0].str.contains("Hill Holder"), 1, 0)  

data["hill_holder"]

In [ ]:
data["light_sensor"] = np.where(temp[0].str.contains("Light sensor"), 1, 0)  

data["light_sensor"]

In [ ]:
data["electrically_heated_windshield"] = np.where(temp[0].str.contains("Electrically heated windshield"), 1, 0)  

data["electrically_heated_windshield"]

In [ ]:
data["electrically_adjustable_seats"] = np.where(temp[0].str.contains("Electrically adjustable seats"), 1, 0)  

data["electrically_adjustable_seats"]

In [ ]:
data["electrical_side_mirrors"] = np.where(temp[0].str.contains("Electrical side mirrors"), 1, 0)  

data["electrical_side_mirrors"]

In [ ]:
data["seat_ventilation"] = np.where(temp[0].str.contains("Seat ventilation"), 1, 0)  

data["seat_ventilation"]

In [ ]:
data["light_sensor"] = np.where(temp[0].str.contains("Light sensor"), 1, 0)  

data["light_sensor"]

In [ ]:
data["rain_sensor"] = np.where(temp[0].str.contains("Rain sensor"), 1, 0)  

data["rain_sensor"]

In [ ]:
data["split_rear_seats"] = np.where(temp[0].str.contains("Split rear seats"), 1, 0)  

data["split_rear_seats"]

In [ ]:
data["electric_tailgate"] = np.where(temp[0].str.contains("Electric tailgate"), 1, 0)  

data["electric_tailgate"]

In [ ]:
data["keyless_central_door_lock"] = np.where(temp[0].str.contains("Keyless central door lock"), 1, 0)  

data["keyless_central_door_lock"]

In [ ]:
data["parking_assist_system_sensors_front"] = np.where(temp[0].str.contains("Parking assist system sensors front"), 1, 0)  

data["parking_assist_system_sensors_front"]

### Entertainment & Media inceleyelim. Bu feature ile ilgili olarak araçlarda burada geçen özelliklerin olup olmadığına göre sütunlar oluşturacağız.

In [139]:
df['\nEntertainment & Media\n'] = [",".join(item) if type(item)==list else item for item in df['\nEntertainment & Media\n']]

data['entertainment'] = df['\nEntertainment & Media\n'].str.extract("(\w.+)").copy()

In [138]:
data['entertainment'].str.get_dummies(sep=",")

,Bluetooth,CD player,Digital radio,Hands-free equipment,MP3,On-board computer,Radio,Sound system,Television,USB
0,1,0,0,1,0,1,1,0,0,0
1,1,0,0,1,0,1,1,1,0,0
2,0,0,0,0,1,1,0,0,0,0
3,1,1,0,1,1,1,1,1,0,1
4,1,1,0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
15914,1,0,1,1,0,1,1,0,0,1
15915,1,0,1,1,0,0,1,0,0,1
15916,1,0,0,1,0,1,1,0,0,0
15917,1,0,1,0,0,0,1,0,0,1


In [139]:
entert = data['entertainment'].str.get_dummies(sep=",")

In [148]:
data.join(entert)   # bunu en son çalıştıralım.

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security,Bluetooth,CD player,Digital radio,Hands-free equipment,MP3,On-board computer,Radio,Sound system,Television,USB
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,3.8,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...","Alloy wheels,Catalytic Con...","ABS,Central door lock,Dayt...",1,0,0,1,0,1,1,0,0,0
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,5.6,7.1,4.7,129,Euro 6,"Air conditioning,Automatic...","Bluetooth,Hands-free equip...","Alloy wheels,Sport seats,S...","ABS,Central door lock,Cent...",1,0,0,1,0,1,1,1,0,0
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,3.8,4.4,3.4,99,Euro 6,"Air conditioning,Cruise co...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Dayt...",0,0,0,0,1,1,0,0,0,0
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,3.8,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Aux...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport seats,V...","ABS,Alarm system,Central d...",1,1,0,1,1,1,1,1,0,1
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,NaN,Diesel,4.1,4.6,3.8,109,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport package...","ABS,Central door lock,Driv...",1,1,0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen","ABS,Central door lock,Cent...",1,0,1,1,0,1,1,0,0,1
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798",4,"1,708",benzine,7.4,9.2,6.3,168,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen,...","ABS,Adaptive Cruise Contro...",1,0,1,1,0,0,1,0,0,1
15916,Renault,Espace,Van,NaN,39875,VAT deductible,15,2019.0,1.0,146.0,Pre-registered,1,Yes,White,NaN,arktis-weiß,Full leather,Black,5,7,NaN,Automatic,"1,997",4,NaN,Diesel,5.3,6.2,4.7,139,Euro 6d-TEMP,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...",Alloy wheels,"ABS,Adaptive Cruise Contro...",1,0,0,1,0,1,1,0,0,0
15917,Renault,Espace,Van,NaN,39700,VAT deductible,10,2019.0,NaN,147.0,Pre-registered,NaN,NaN,Grey,NaN,Grigio,Part leather,NaN,5,7,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,Euro 6,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ra...","Alloy wheels,Touch screen","ABS,Blind spot monitor,Dri...",1,0,1,0,0,0,1,0,0,1


### 2. yöntem olarak herbir eleman için olanlara 1 olmayanlara 0 atayarak da aynı sonuca ulaşabiliriz.

In [298]:
data["bluetooth"] = np.where(temp[0].str.contains("Bluetooth"), 1, 0)  

data["bluetooth"]

0        1
1        1
2        0
3        1
4        1
        ..
15914    1
15915    1
15916    1
15917    1
15918    0
Name: Bluetooth, Length: 15919, dtype: int32

In [ ]:
data["handsfree_equipment"] = np.where(temp[0].str.contains("Hands-free equipment"), 1, 0)  

data["handsfree_equipment"]

In [ ]:
data["onboard_computer"] = np.where(temp[0].str.contains("On-board computer"), 1, 0)  

data["onboard_computer"]

In [ ]:
data["radio"] = np.where(temp[0].str.contains("Radio"), 1, 0)  

data["radio"]

In [ ]:
data["CD_player"] = np.where(temp[0].str.contains("CD player"), 1, 0)  

data["CD_player"] 

In [ ]:
data["television"] = np.where(temp[0].str.contains("Television"), 1, 0)  

data["television"] 

In [ ]:
data["digital_radio"] = np.where(temp[0].str.contains("Digital radio"), 1, 0)  

data["digital_radio"] 

In [ ]:
data["MP3"] = np.where(temp[0].str.contains("MP3"), 1, 0)  

data["MP3"] 

In [ ]:
data["sound_system"] = np.where(temp[0].str.contains("Sound system"), 1, 0)  

data["sound_system"] 

In [ ]:
data["USB"] = np.where(temp[0].str.contains("USB"), 1, 0)  

data["USB"]

### Extras inceleyelim.

In [140]:
df['\nExtras\n'] = [",".join(item) if type(item)==list else item for item in df['\nExtras\n']]

data['Extras'] = df['\nExtras\n'].str.extract("(\w.+)").copy()

In [141]:
data['Extras'].str.get_dummies(sep=",")

,Alloy wheels,Cab or rented Car,Catalytic Converter,Handicapped enabled,Right hand drive,Roof rack,Shift paddles,Ski bag,Sliding door,Sport package,Sport seats,Sport suspension,Touch screen,Trailer hitch,Tuned car,Voice Control,Winter tyres
0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
15915,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
15916,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15917,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [142]:
extra = data['Extras'].str.get_dummies(sep=",")

data.join(extra )  

In [149]:
data.join(extra )   # bunu en son çalıştıralım.

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security,Alloy wheels,Cab or rented Car,Catalytic Converter,Handicapped enabled,Right hand drive,Roof rack,Shift paddles,Ski bag,Sliding door,Sport package,Sport seats,Sport suspension,Touch screen,Trailer hitch,Tuned car,Voice Control,Winter tyres
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,3.8,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...","Alloy wheels,Catalytic Con...","ABS,Central door lock,Dayt...",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,5.6,7.1,4.7,129,Euro 6,"Air conditioning,Automatic...","Bluetooth,Hands-free equip...","Alloy wheels,Sport seats,S...","ABS,Central door lock,Cent...",1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,3.8,4.4,3.4,99,Euro 6,"Air conditioning,Cruise co...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Dayt...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,3.8,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Aux...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport seats,V...","ABS,Alarm system,Central d...",1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,NaN,Diesel,4.1,4.6,3.8,109,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport package...","ABS,Central door lock,Driv...",1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen","ABS,Central door lock,Cent...",1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798",4,"1,708",benzine,7.4,9.2,6.3,168,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen,...","ABS,Adaptive Cruise Contro...",1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
15916,Renault,Espace,Van,NaN,39875,VAT deductible,15,2019.0,1.0,146.0,Pre-registered,1,Yes,White,NaN,arktis-weiß,Full leather,Black,5,7,NaN,Automatic,"1,997",4,NaN,Diesel,5.3,6.2,4.7,139,Euro 6d-TEMP,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...",Alloy wheels,"ABS,Adaptive Cruise Contro...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15917,Renault,Espace,Van,NaN,39700,VAT deductible,10,2019.0,NaN,147.0,Pre-registered,NaN,NaN,Grey,NaN,Grigio,Part leather,NaN,5,7,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,Euro 6,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ra...","Alloy wheels,Touch screen","ABS,Blind spot monitor,Dri...",1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


### 2. yöntem olarak herbir eleman için olanlara 1 olmayanlara 0 atayarak da aynı sonuca ulaşabiliriz.

In [300]:
data["alloy_wheels"] = np.where(temp[0].str.contains("Alloy wheels"), 1, 0)  

data["alloy_wheels"]

0        1
1        1
2        1
3        1
4        1
        ..
15914    1
15915    1
15916    1
15917    1
15918    1
Name: alloy_wheels, Length: 15919, dtype: int32

In [ ]:
data["touch_screen"] = np.where(temp[0].str.contains("Touch screen"), 1, 0)  

data["touch_screen"]

In [ ]:
data["voice_control"] = np.where(temp[0].str.contains("Voice Control"), 1, 0)  

data["voice_control"]

In [ ]:
data["winter_tyres"] = np.where(temp[0].str.contains("Winter tyres"), 1, 0)  

data["winter_tyres"]

In [ ]:
data["roof_rack"] = np.where(temp[0].str.contains("Roof rack"), 1, 0)  

data["roof_rack"]

In [ ]:
data["sport_package"] = np.where(temp[0].str.contains("Sport package"), 1, 0)  

data["sport_package"]

In [ ]:
data["catalytic_converter"] = np.where(temp[0].str.contains("Catalytic Converter"), 1, 0)  

data["catalytic_converter"]

In [ ]:
data["ski_bag"] = np.where(temp[0].str.contains("Ski bag"), 1, 0)  

data["ski_bag"]

In [ ]:
data["sport_suspension"] = np.where(temp[0].str.contains("Sport suspension"), 1, 0)  

data["sport_suspension"]

In [ ]:
data["trailer_hitch"] = np.where(temp[0].str.contains("Trailer hitch"), 1, 0)  

data["trailer_hitch"]

In [ ]:
data["shift_paddles"] = np.where(temp[0].str.contains("Shift paddles"), 1, 0)  

data["shift_paddles"]

### Safety & Security inceleyelim.

In [141]:
df['\nSafety & Security\n'] = [",".join(item) if type(item)==list else item for item in df['\nSafety & Security\n']]

data['safety_security'] = df['\nSafety & Security\n'].str.extract("(\w.+)").copy()

In [144]:
safety = data['safety_security'].str.get_dummies(sep=",")

In [150]:
data.join(safety)     # bunu en son çalıştıralım.

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security,ABS,Adaptive Cruise Control,Adaptive headlights,Alarm system,Blind spot monitor,Central door lock,Central door lock with remote control,Daytime running lights,Driver drowsiness detection,Driver-side airbag,Electronic stability control,Emergency brake assistant,Emergency system,Fog lights,Head airbag,Immobilizer,Isofix,LED Daytime Running Lights,LED Headlights,Lane departure warning system,Night view assist,Passenger-side airbag,Power steering,Rear airbag,Side airbag,Tire pressure monitoring system,Traction control,Traffic sign recognition,Xenon headlights
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,3.8,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...","Alloy wheels,Catalytic Con...","ABS,Central door lock,Dayt...",1,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,1
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,5.6,7.1,4.7,129,Euro 6,"Air conditioning,Automatic...","Bluetooth,Hands-free equip...","Alloy wheels,Sport seats,S...","ABS,Central door lock,Cent...",1,0,0,0,0,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,0,1
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,3.8,4.4,3.4,99,Euro 6,"Air conditioning,Cruise co...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Dayt...",1,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,3.8,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Aux...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport seats,V...","ABS,Alarm system,Central d...",1,0,0,1,0,0,1,0,1,1,1,0,1,0,1,1,1,0,0,0,0,1,1,0,1,1,0,0,0
4,Audi,A1,Sedans,1.4,16790,None,16200,2016.0,1.0,66.0,Used,1,Yes,Black,Metallic,Mythosschwarz Metallic,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,NaN,Diesel,4.1,4.6,3.8,109,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport package...","ABS,Central door lock,Driv...",1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault,Espace,Van,NaN,39950,VAT deductible,,NaN,NaN,147.0,New,NaN,NaN,Grey,Metallic,Grigio scuro,NaN,NaN,5,5,NaN,Automatic,"1,997",4,"1,758",Diesel,5.3,6.2,4.7,139,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen","ABS,Central door lock,Cent...",1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0
15915,Renault,Espace,Van,NaN,39885,VAT deductible,9900,2019.0,1.0,165.0,Used,1,NaN,Grey,Metallic,Stahl-Grau,Cloth,NaN,5,5,0000/000,Automatic,"1,798",4,"1,708",benzine,7.4,9.2,6.3,168,NaN,"Air conditioning,Automatic...","Bluetooth,Digital radio,Ha...","Alloy wheels,Touch screen,...","ABS,Adaptive Cruise Contro...",1,1,0,0,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,0
15916,Renault,Espace,Van,NaN,39875,VAT deductible,15,2019.0,1.0,146.0,Pre-registered,1,Yes,White,NaN,arktis-weiß,Full leather,Black,5,7,NaN,Automatic,"1,997",4,NaN,Diesel,5.3,6.2,4.7,139,Euro 6d-TEMP,"Air conditioning,Armrest,A...","Bluetooth,Hands

### 2. yöntem olarak herbir eleman için olanlara 1 olmayanlara 0 atayarak da aynı sonuca ulaşabiliriz.

In [ ]:
data["ABS"] = np.where(temp[0].str.contains("ABS"), 1, 0)  

data["ABS"]

In [ ]:
data["central_door_lock"] = np.where(temp[0].str.contains("Central door lock"), 1, 0)  

data["central_door_lock"]

In [ ]:
data["daytime_running_lights"] = np.where(temp[0].str.contains("Daytime running lights"), 1, 0)  

data["daytime_running_lights"]

In [ ]:
data["driverside_airbag"] = np.where(temp[0].str.contains("Driver-side airbag"), 1, 0)  

data["driverside_airbag"]

In [ ]:
data["fog_lights"] = np.where(temp[0].str.contains("Fog lights"), 1, 0)  

data["fog_lights"]

In [ ]:
data["xenon_headlights"] = np.where(temp[0].str.contains("Xenon headlights"), 1, 0)  

data["xenon_headlights"]

In [ ]:
data["immobilizer"] = np.where(temp[0].str.contains("Immobilizer"), 1, 0)  

data["immobilizer"]

In [ ]:
data["isofix"] = np.where(temp[0].str.contains("Isofix"), 1, 0)  

data["isofix"]

In [ ]:
data["passengerside_airbag"] = np.where(temp[0].str.contains("Passenger-side airbag"), 1, 0)  

data["passengerside_airbag"]

In [ ]:
data["power_steering"] = np.where(temp[0].str.contains("Power steering"), 1, 0)  

data["power_steering"]

In [ ]:
data["side_airbag"] = np.where(temp[0].str.contains("Side airbag"), 1, 0)  

data["side_airbag"]

In [ ]:
data["tire_pressure_monitoring_system"] = np.where(temp[0].str.contains("Tire pressure monitoring system"), 1, 0)  

data["tire_pressure_monitoring_system"]

In [ ]:
data["traction_control"] = np.where(temp[0].str.contains("Traction control"), 1, 0)  

data["traction_control"]

### Description inceleyelim. Bu sütundaki bilgilerin hem dil olarak hem de içerik olarak çok karmaşık bir yapısı olduğundan dolayı bu sütunu düşeceğiz.

### Emission Label inceleyelim. 

In [142]:
temp = df['Emission Label'].copy()    # 11974 eksik veri var. Buda çok yüksek bir rakam. Marka model registration dan doldurulabilir.

In [231]:
data["emission_label"] = temp.apply(pd.Series)[0].str.extract("(\S+)")

In [233]:
data["emission_label"].value_counts(dropna=False)

NaN    11974
4       3553
1        381
5          8
3          2
2          1
Name: emission_label, dtype: int64

### Gears inceleyelim. 

In [143]:
temp = df['Gears'].copy()    # 4712 veri eksik.

In [144]:
data["gears"] = temp.apply(pd.Series)[0].str.extract("(\d+)") 

In [145]:
data["gears"].value_counts(dropna=False)

6      5822
NaN    4712
5      3239
7      1908
8       224
9         6
1         2
3         2
4         2
50        1
2         1
Name: gears, dtype: int64

### Country version inceleyelim. 

In [146]:
temp = df['Country version'].copy()   # 8333 eksik veri var. 

In [147]:
data["country"] = temp.apply(pd.Series)[0].str.extract("(\w+)") 

In [148]:
data["country"].value_counts(dropna=False)

NaN            8333
Germany        4502
Italy          1038
European        507
Netherlands     464
Spain           325
Belgium         314
Austria         208
Czech            52
Poland           49
France           38
Denmark          33
Hungary          28
Japan             8
Croatia           4
Slovakia          4
Sweden            3
Bulgaria          2
Romania           2
Serbia            1
Egypt             1
Slovenia          1
Luxembourg        1
Switzerland       1
Name: country, dtype: int64

In [149]:
data.drop("country", axis=1, inplace=True)

### Electricity consumption inceleyelim. 

In [248]:
temp = df['Electricity consumption'].copy()   # 137 veri var. Kullanışlı değil. Bu sütunu sileceğiz. 

In [249]:
data["cons_elec"] = temp.apply(pd.Series)[0].str.extract("(\d+)") 

In [250]:
data["cons_elec"].value_counts(dropna=False)

NaN    15782
0        137
Name: cons_elec, dtype: int64

### Last Service Date inceleyelim. 

In [251]:
temp = df['Last Service Date'].copy()   # 15353 eksik veri var. Sileceğiz.

In [252]:
data["Last_Serv"] = temp.apply(pd.Series)[0].str.extract('(\d{4})')

In [253]:
data["Last_Serv"].value_counts(dropna=False)

NaN     15353
2019      316
2018      216
2017       28
2016        6
Name: Last_Serv, dtype: int64

### Other Fuel Types inceleyelim. Veri içeriği boş ve silinecek.

In [254]:
temp = df['Other Fuel Types'].copy()

### Availability inceleyelim. 

In [266]:
temp = df['Availability'].copy()  # 15284 eksik veri var. silinecek.

In [267]:
data["availability"] = temp.apply(pd.Series)[0].str.extract('(\d{1,3})') 

In [268]:
data["availability"].value_counts(dropna=False)

NaN    15284
90       196
120      182
1         51
3         35
5         35
14        24
180       24
7         20
150       18
2         16
60        13
42        10
21         8
4          2
6          1
Name: availability, dtype: int64

### Last Timing Belt Service Date inceleyelim. 

In [269]:
temp = df['Last Timing Belt Service Date'].copy()   # 15903 veri eksik. Düşelim. 

In [271]:
data["belt_serv"] = temp.apply(pd.Series)[0].str.extract('(\d{4})')

In [273]:
data["belt_serv"] .value_counts(dropna=False)

NaN     15903
2018        5
2019        5
1900        2
2016        2
1970        1
2017        1
Name: belt_serv, dtype: int64

### Available from inceleyelim. Bu sütunun ne işe yaradığını anlamadım. 

In [162]:
temp = df['Available from'].copy()  # 15647 eksik veri var.

In [163]:
data["available"] = temp.apply(pd.Series)[0].str.extract('(\d{2}/\d{2}/\d{2})')

In [164]:
data["available"].value_counts(dropna=False)

NaN         15647
31/08/19       98
30/07/19       25
15/08/19       24
16/09/19       14
29/06/19       12
01/07/19       11
28/06/19       10
19/08/19        7
10/09/19        6
27/06/19        5
10/11/19        5
26/06/19        5
08/07/19        4
01/08/19        3
25/10/19        3
15/07/19        3
05/07/19        2
20/07/19        2
30/06/19        2
04/07/19        2
05/08/19        2
24/08/19        2
16/07/19        2
30/09/19        1
18/08/19        1
03/08/19        1
29/09/19        1
29/07/19        1
06/12/19        1
02/07/19        1
16/08/19        1
10/10/19        1
01/09/19        1
27/07/19        1
18/07/19        1
31/10/19        1
24/09/19        1
17/08/19        1
14/09/19        1
10/12/19        1
19/07/19        1
11/08/19        1
24/07/19        1
05/12/19        1
10/07/19        1
03/12/19        1
Name: available, dtype: int64

In [145]:
data.head(4)

,make,model,body_type,cc,price,vat,km,registration,prev_owner,power_hp,vehicle_condition,previous_owners,Inspection_new,body_color,paintwork,body_color_original,upholstery,interior_color,door_nr,seat_nr,model_code,gear,displacement,cylinders,weight,fuel,consumption_comb,consumption_city,consumption_country,CO2_emission,emission_class,comfort_convenience,entertainment,Extras,safety_security
0,Audi,A1,Sedans,1.4,15770,VAT deductible,56013,2016.0,2.0,66.0,Used,2,Yes,Black,Metallic,Mythosschwarz,Cloth,Black,5,5,0588/BDF,Automatic,"1,422",3,"1,220",Diesel,3.8,4.3,3.5,99,Euro 6,"Air conditioning,Armrest,A...","Bluetooth,Hands-free equip...","Alloy wheels,Catalytic Con...","ABS,Central door lock,Dayt..."
1,Audi,A1,Sedans,1.8,14500,Price negotiable,80000,2017.0,NaN,141.0,Used,NaN,NaN,Red,NaN,NaN,Cloth,Grey,3,4,0588/BCY,Automatic,"1,798",4,"1,255",benzine,5.6,7.1,4.7,129,Euro 6,"Air conditioning,Automatic...","Bluetooth,Hands-free equip...","Alloy wheels,Sport seats,S...","ABS,Central door lock,Cent..."
2,Audi,A1,Sedans,1.6,14640,VAT deductible,83450,2016.0,1.0,85.0,Used,1,NaN,Black,Metallic,mythosschwarz metallic,Cloth,Black,4,4,NaN,Automatic,"1,598",NaN,NaN,Diesel,3.8,4.4,3.4,99,Euro 6,"Air conditioning,Cruise co...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Dayt..."
3,Audi,A1,Sedans,1.4,14500,None,73000,2016.0,1.0,66.0,Used,1,NaN,Brown,Metallic,NaN,NaN,NaN,3,4,NaN,Automatic,"1,422",3,"1,195",Diesel,3.8,4.3,3.5,99,Euro 6,"Air suspension,Armrest,Aux...","Bluetooth,CD player,Hands-...","Alloy wheels,Sport seats,V...","ABS,Alarm system,Central d..."


In [150]:
data.to_csv("Autoscout_data_cleaning.csv", index=False)